sources:

https://www.azoft.com/blog/ocr-embossed-text/

https://stackoverflow.com/questions/9398374/prepare-complex-image-for-ocr

In [13]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [14]:
# image_name = "tire_interpolation-5.png"
# image_name = "hole_tire.jpg"
image_name = "hole_tire_colored.jpg"

image_extension = image_name.split(".")[-1]
image_name = image_name.split(".")[0]

image = cv2.imread("./in/"+image_name+"."+image_extension)

Step 1. Convert the source image I to grayscale.

In [15]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imwrite("./out/paper_approach/"+image_name+"_Step1-gray."+image_extension, gray)

True

Step 2. Detect edges in the grayscale image using the Sobel or similar operator. The resulting image we denote by Ie.

In [16]:
temp = gray
temp = cv2.GaussianBlur(temp, (3, 3), 0)

gx = cv2.Sobel(temp, cv2.CV_64F, 1, 0, ksize=5)
gy = cv2.Sobel(temp, cv2.CV_64F, 0, 1, ksize=5)

abs_grad_x = cv2.convertScaleAbs(gx)
abs_grad_y = cv2.convertScaleAbs(gy)

Ie = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
cv2.imwrite("./out/paper_approach/"+image_name+"_Step2-sobel."+image_extension, Ie)

Ie = Ie.astype(np.uint8)

Step 3. Perform binarization of Ie using Otsu’s method. The resulting image we denote by Ib.

In [17]:
T, Ib = cv2.threshold(Ie, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
print("Threshold: ", T)
cv2.imwrite("./out/paper_approach/"+image_name+"_Step3-OtsuThreshold."+image_extension, Ib)

Threshold:  124.0


True

Step 4. Create a 2-dimensional array S with the same dimensions as I and fill it with zeroes.

In [53]:
S = np.zeros(Ib.shape, dtype=np.uint8)

Step 5. Create a binary mask W_in and binary mask W_out. Their dimensions should be N_in×N_in and N_out×N_out respectively. N_in and N_out values depend on the stroke width in the image and N_in is always less than or equal to N_out.